## Imports


In [ ]:
import os
import json
import pandas as pd
import psycopg2
from psycopg2 import OperationalError
from psycopg2 import ProgrammingError


## Data Connection


In [ ]:
# read user file
with open("package/account/user.json", "r") as read_file:
    user = json.load(read_file)


In [ ]:
# make discord name
discord_username_full = user["username"] + "#" + str(user["discriminator"])


In [ ]:
# function to connect to postgresql db
def create_connection(db_name, db_user, db_password, db_host, db_port):
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        connection = None
        print(f"The error '{e}' occurred")
        print("Error Type:", type(e))

    return connection


In [ ]:
def read_secrets() -> dict:
    filename = os.path.join("secrets.json")
    try:
        with open(filename, mode="r") as f:
            return json.loads(f.read())
    except FileNotFoundError:
        return {}


In [ ]:
secrets = read_secrets()

In [ ]:
# set up database connection
con = create_connection(
    secrets["db_name"],
    secrets["db_user"],
    secrets["db_password"],
    secrets["db_host"],
    secrets["db_port"],
)

In [ ]:
cur = con.cursor()

In [ ]:
# function to select data, returns a dataframe
def select(query):
    data_df = None
    try:
        cur.execute(query)
        col_names = [desc[0] for desc in cur.description]
        data_df = pd.DataFrame(data=cur.fetchall(), columns=col_names)
    except ProgrammingError as e:
        con.rollback()
        print(f"The error '{e}' occurred")
        print("Error Type:", type(e))

    return data_df


## Import Data


In [ ]:
select_query = """
SELECT *
FROM discord_data.dim_channel
"""

df_dim_channel = select(select_query)

In [ ]:
select_query = """
SELECT *
FROM discord_data.dim_channel_type
"""

df_dim_channel_type = select(select_query)

In [ ]:
select_query = """
SELECT *
FROM discord_data.dim_server
"""

df_dim_server = select(select_query)

In [ ]:
select_query = """
SELECT *
FROM discord_data.fact_messages
"""

df_fact_messages = select(select_query)

In [ ]:
select_query = """
SELECT *
FROM discord_data.dim_channel dc
LEFT JOIN discord_data.dim_channeL_type dct
on dc.channel_type_key = dct.channel_type_key
LEFT JOIN discord_data.dim_server ds
on dc.server_id = ds.server_id
LEFT JOIN discord_data.fact_messages fm
ON fm.channel_id = dc.channel_id
"""

df = select(select_query)

In [ ]:
df.head()


In [ ]:
# read from csv (due to it being semi structured data)
# may transition to a NoSQL database in the future
df_fact_activity = pd.read_csv("fact_activity.csv")


In [ ]:
df_fact_activity.head()
